In [106]:
import os
import sys
sys.path.append('../../')  # subir 1 nivel desde notebooks a src
from config.paths import SRC_DIR
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [107]:
base = pd.read_csv(r"C:\Users\JTRUJILLO\Documents\Galgo\Scripts\Otros\deep_research_models\src\data\db\bd_comparador.csv")

In [108]:
pais = "Colombia"

In [109]:
co_inventory = r"C:\Users\JTRUJILLO\Documents\Galgo\Scripts\Data\historical_data\history\CO\BaseCOv2.csv"
mx_inventory = r"C:\Users\JTRUJILLO\Documents\Galgo\Scripts\Data\historical_data\history\MX\BaseMXv5.csv"
cl_inventory = r"C:\Users\JTRUJILLO\Documents\Galgo\Scripts\Data\historical_data\history\CL-MOTORCYCLES\df_cl-motorcycles_history.csv"

In [110]:
if pais == "Colombia":
    base_data = pd.read_csv(co_inventory)
    country_code = "CO"
elif pais == "Mexico":
    base_data = pd.read_csv(mx_inventory)
    country_code = "MX"
elif pais == "Chile":
    base_data = pd.read_csv(cl_inventory)
    country_code = "CL"

C:\Users\JTRUJILLO\AppData\Local\Temp\ipykernel_1396\1272235273.py:2: DtypeWarning: Columns (8,23,30) have mixed types. Specify dtype option on import or set low_memory=False.
  base_data = pd.read_csv(co_inventory)


In [111]:
# Filtrar por fecha de actualización
filtro_fecha = base_data["Fecha de actualización"] == "22/12/2025"
base_filtrada = base_data[filtro_fecha]

# Filtrar por estado disponible o sin stock
estados_validos = ["available", "no_stock"]
base_filtrada = base_filtrada[base_filtrada["Estado"].isin(estados_validos)]

# Excluir los que tienen redirección
base_filtrada = base_filtrada[base_filtrada["tiene_redirección"] == False]

# Seleccionar solo las columnas necesarias
columnas_a_usar = ["Fecha de actualización", "code", "Marca", "Modelo", "Año", "Precio Neto", "Tipo", "Estado"]
df_columnas_seleccionadas = base_filtrada[columnas_a_usar].copy()
df_columnas_seleccionadas = df_columnas_seleccionadas.rename(columns={"Precio Neto": "Precio"})

# Eliminar duplicados por código
df_desduplicadas = df_columnas_seleccionadas.drop_duplicates(subset="code").copy()

# Agregar columna de URL
df_desduplicadas["url"] = df_desduplicadas["code"].apply(
    lambda x: f"https://www.galgo.com/{country_code.lower()}/motos/{x}"
)

# Definir columnas finales
columnas_finales = ["Fecha de actualización", "code", "Marca", "Modelo", "Año", "Tipo", "Precio", "url", "Estado"]
df_columnas_finales = df_desduplicadas[columnas_finales]

In [112]:
# Limpieza sencilla del dataframe: elimino espacios y saltos de línea en columnas y valores

# Eliminar espacios y saltos de línea en los nombres de las columnas
base.columns = base.columns.str.strip()

# Eliminar espacios y saltos de línea en todos los valores string del dataframe
base = base.applymap(lambda x: x.strip() if isinstance(x, str) else x)

base.head()

C:\Users\JTRUJILLO\AppData\Local\Temp\ipykernel_1396\1463688563.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  base = base.applymap(lambda x: x.strip() if isinstance(x, str) else x)


,Label_1,Metric_1
0,https://www.galgo.com/co/comparador/motos/CO1783-suzuki-dr150-modelo-2025-vs-CO1418-yamaha-xtz-150-2024,2794.0
1,https://www.galgo.com/co/comparador/motos/CO1894-honda-xr-190l-vs-CO730-honda-xre-190-dlx,2676.0
2,https://www.galgo.com/co/comparador/motos/CO728-honda-xr150l-vs-CO1757-suzuki-dr-150-fi-abs-2025,2543.0
3,https://www.galgo.com/co/comparador/motos/CO1894-honda-xr-190l-vs-CO1029-yamaha-xtz-150,2417.0
4,https://www.galgo.com/co/comparador/motos/CO2961-hero-hunk-125-r-vs-CO731-tvs-raider-125,2291.0


In [113]:
base.rename(columns = {"Metric_1": "Views", "Label_1": "url"}, inplace = True)

In [114]:
base["Views"] = base["Views"].astype(int)

In [115]:
# Extraer códigos completos de las URLs y verificar si están disponibles
import re
import json

# Obtener códigos disponibles en df_columnas_finales
codigos_disponibles = set(df_columnas_finales['code'].unique())

def verificar_stock(url):
    # Extraer los códigos completos de la URL
    # Ejemplo: .../motos/CO2961-hero-hunk-125-r-vs-CO731-tvs-raider-125
    # Código 1 completo: CO2961-hero-hunk-125-r
    # Código 2 completo: CO731-tvs-raider-125
    match = re.search(r'/motos/(.+?)-vs-(.+?)$', url)
    if match:
        codigo1 = match.group(1).rstrip('/')  # Código completo antes del "-vs-"
        codigo2 = match.group(2).rstrip('/')  # Código completo después del "-vs-"
        # Solo disponible si AMBOS códigos están disponibles
        if codigo1 in codigos_disponibles and codigo2 in codigos_disponibles:
            return "disponible"
    return "no_disponible"

def generar_json_stock(url):
    # Generar JSON con el estado de disponibilidad de cada código
    match = re.search(r'/motos/(.+?)-vs-(.+?)$', url)
    if match:
        codigo1 = match.group(1).rstrip('/')
        codigo2 = match.group(2).rstrip('/')
        estado_codigo1 = "disponible" if codigo1 in codigos_disponibles else "no_disponible"
        estado_codigo2 = "disponible" if codigo2 in codigos_disponibles else "no_disponible"
        return json.dumps({
            codigo1: estado_codigo1,
            codigo2: estado_codigo2
        }, ensure_ascii=False)
    return json.dumps({}, ensure_ascii=False)

base['stock'] = base['url'].apply(verificar_stock)
base['stock_detail'] = base['url'].apply(generar_json_stock)

In [119]:
base[(base["url"].str.contains("CO2961-hero-hunk-125-r")) & (base["stock"] == "disponible")].head(3)

,url,Views,stock,stock_detail
4,https://www.galgo.com/co/comparador/motos/CO2961-hero-hunk-125-r-vs-CO731-tvs-raider-125,2291,disponible,"{""CO2961-hero-hunk-125-r"": ""disponible"", ""CO731-tvs-raider-125"": ""disponible""}"
19,https://www.galgo.com/co/comparador/motos/CO2961-hero-hunk-125-r-vs-CO2861-tvs-raider-fi-125,1272,disponible,"{""CO2961-hero-hunk-125-r"": ""disponible"", ""CO2861-tvs-raider-fi-125"": ""disponible""}"
146,https://www.galgo.com/co/comparador/motos/CO2961-hero-hunk-125-r-vs-CO2615-hero-hunk-150-xt,466,disponible,"{""CO2961-hero-hunk-125-r"": ""disponible"", ""CO2615-hero-hunk-150-xt"": ""disponible""}"


In [121]:
base.to_csv(f"{SRC_DIR}/data/db/bd_comparador_stock.csv", index=False)